# Regressiemodel

In [13]:
from datetime import datetime
import pandas as pd
#from sklearn.model_selection import train_test_split
#from sklearn import linear_model
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_squared_error, mean_absolute_error
import pyodbc
import sqlite3
import sqlalchemy
import matplotlib.pyplot as plt

Gegevens ophalen uit Data Warehouse

In [14]:
DB = {'servername': 'ENVY-AKRAM\\SQLEXPRESS','database': 'Great Outdoors - Akram en Sylvan'}
conn_str = ('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_conn = pyodbc.connect(conn_str, timeout=120)
export_cursor = export_conn.cursor()

engine = sqlalchemy.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")

print(export_cursor.execute("SELECT @@version;"))
export_cursor

In [24]:
order_dw = pd.read_sql_query("SELECT * FROM [Order]", engine)
order_dw

,ORDER_NUMBER,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,SALES_STAFF_CODE,SALES_BRANCH_CODE,ORDER_DATE,ORDER_METHOD_CODE,ORDER_DETAIL_CODE,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,ORDER_TURNOVER,ORDER_PROFIT
0,6773,135,146,60,20,2021-12-05,1,77552,77,44,45.31,90.63,90.63,3987.72,1994.08
1,6773,135,146,60,20,2021-12-05,1,77590,82,38,20.47,30.71,30.71,1166.98,389.12
2,6773,135,146,60,20,2021-12-05,1,77667,85,14,177.17,301.2,301.2,4216.80,1736.42
3,6773,135,146,60,20,2021-12-05,1,77813,90,152,2.47,6.92,6.92,1051.84,676.40
4,6773,135,146,60,20,2021-12-05,1,77863,92,20,1.97,4.92,4.92,98.40,59.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,6773,135,146,60,20,2021-12-05,1,76804,51,28,34.47,68.94,68.94,1930.32,965.16
43059,6773,135,146,60,20,2021-12-05,1,76980,57,20,38.97,77.94,77.94,1558.80,779.40
43060,6773,135,146,60,20,2021-12-05,1,77003,58,14,56.88,79.65,79.65,1115.10,318.78
43061,6773,135,146,60,20,2021-12-05,1,77043,60,22,9.89,19.78,19.78,435.16,217.58


In [23]:
product_dw = pd.read_sql_query("SELECT * FROM Product", engine)
product_dw

,PRODUCT_NUMBER,PRODUCT_NAME,PRODUCT_IMAGE,DESCRIPTION,INTRODUCTION_DATE,PRODUCTION_COST,MARGIN,LANGUAGE,PRODUCT_TYPE_CODE,PRODUCT_TYPE_EN,PRODUCT_LINE_CODE,PRODUCT_LINE_EN
0,1,TrailChef Water Bag,P01CE1CG1.jpg,"Lightweight, collapsible bag to carry liquids ...",15-2-2011,4,.33,EN,1,Cooking Gear,1,Camping Equipment
1,10,TrailChef Utensils,P10CE1CG1.jpg,"Spoon, fork and knife set made of a light yet ...",15-2-2011,10,.4,EN,1,Cooking Gear,1,Camping Equipment
2,100,Insect Bite Relief,P100OP4FA17.jpg,The Insect Bite Relief helps the itching and s...,15-2-2011,3,.5,EN,17,First Aid,4,Outdoor Protection
3,101,Hailstorm Steel Irons,P101GE5IR18.jpg,Iron is 17-4 stainless steel. Shafts are grap...,15-12-2019,305.54,.43,EN,18,Irons,5,Golf Equipment
4,102,Hailstorm Titanium Irons,P102GE5IR18.jpg,Made entirely of pure titanium. The ultimate i...,10-12-2019,380.95,.51,EN,18,Irons,5,Golf Equipment
...,...,...,...,...,...,...,...,...,...,...,...,...
110,95,Sun Shield,P91OP4SS16.jpg,"PABA free sunscreen, SPF 30, poison oak and iv...",15-2-2011,3,.5,EN,16,Sunscreen,4,Outdoor Protection
111,96,Compact Relief Kit,P96OP4FA17.jpg,A personal first aid kit is recommended for ev...,15-2-2011,16.43,.28,EN,17,First Aid,4,Outdoor Protection
112,97,Deluxe Family Relief Kit,P96OP4FA17.jpg,A complete medical kit suitable for families w...,5-3-2013,25,.28,EN,17,First Aid,4,Outdoor Protection
113,98,Calamine Relief,P98OP4FA17.jpg,Use the Calamine Relief for allergic skin reac...,15-2-2011,3,.5,EN,17,First Aid,4,Outdoor Protection


Dataframe opstellen

In [25]:
datasetPrepProduct = pd.merge(
    order_dw[['PRODUCT_NUMBER', 'QUANTITY', 'UNIT_COST']],
    product_dw[['PRODUCT_NUMBER', 'INTRODUCTION_DATE', 'PRODUCT_TYPE_CODE', 'PRODUCT_LINE_CODE']],
    on='PRODUCT_NUMBER', how='inner'
)
datasetPrepProduct['UNIT_COST'] = datasetPrepProduct['UNIT_COST'].astype(float)
datasetPrepProduct

,PRODUCT_NUMBER,QUANTITY,UNIT_COST,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCT_LINE_CODE
0,77,44,45.31,26-10-2016,12,3
1,82,38,20.47,5-3-2013,14,3
2,85,14,177.17,12-6-2015,14,3
3,90,152,2.47,15-2-2011,15,4
4,92,20,1.97,15-2-2011,16,4
...,...,...,...,...,...,...
43058,51,28,34.47,10-1-2021,8,2
43059,57,20,38.97,10-1-2021,9,2
43060,58,14,56.88,10-1-2021,9,2
43061,60,22,9.89,10-1-2021,9,2


In [26]:
#helperfunctie

def ToProductAge(date_str):
    date_obj = pd.to_datetime(date_str, dayfirst=True)
    return (datetime.now() - date_obj).days


In [27]:
datasetProduct = datasetPrepProduct.groupby('PRODUCT_NUMBER').agg({
    'QUANTITY': 'sum',
    'INTRODUCTION_DATE': 'first',
    'UNIT_COST': 'mean',
    'PRODUCT_TYPE_CODE': 'first',
    'PRODUCT_LINE_CODE': 'first'
}).reset_index()
datasetProduct

,PRODUCT_NUMBER,QUANTITY,INTRODUCTION_DATE,UNIT_COST,PRODUCT_TYPE_CODE,PRODUCT_LINE_CODE
0,1,41770,15-2-2011,4.113607,1,1
1,10,15928,15-2-2011,10.429155,1,1
2,100,29980,15-2-2011,2.848523,17,4
3,101,5764,15-12-2019,316.939286,18,5
4,102,5014,10-12-2019,402.653541,18,5
...,...,...,...,...,...,...
110,95,53086,15-2-2011,2.875822,16,4
111,96,5974,15-2-2011,16.269130,17,4
112,97,3246,5-3-2013,24.459522,17,4
113,98,18362,15-2-2011,2.897125,17,4


Variabelen numeriek maken

In [28]:
datasetProduct

,PRODUCT_NUMBER,QUANTITY,INTRODUCTION_DATE,UNIT_COST,PRODUCT_TYPE_CODE,PRODUCT_LINE_CODE
0,1,41770,15-2-2011,4.113607,1,1
1,10,15928,15-2-2011,10.429155,1,1
2,100,29980,15-2-2011,2.848523,17,4
3,101,5764,15-12-2019,316.939286,18,5
4,102,5014,10-12-2019,402.653541,18,5
...,...,...,...,...,...,...
110,95,53086,15-2-2011,2.875822,16,4
111,96,5974,15-2-2011,16.269130,17,4
112,97,3246,5-3-2013,24.459522,17,4
113,98,18362,15-2-2011,2.897125,17,4


In [29]:
datasetProduct['DAYS_SINCE_INTRODUCTION'] = datasetProduct['INTRODUCTION_DATE'].apply(ToProductAge)
datasetProduct.drop('INTRODUCTION_DATE', axis=1, inplace=True)


typeDummies = pd.get_dummies(datasetProduct['PRODUCT_TYPE_CODE'])
datasetProduct = pd.concat([datasetProduct, typeDummies], axis=1)
datasetProduct.drop('PRODUCT_TYPE_CODE', axis=1, inplace=True)

lineDummies = pd.get_dummies(datasetProduct['PRODUCT_LINE_CODE'])
datasetProduct = pd.concat([datasetProduct, lineDummies], axis = 1)
datasetProduct.drop('PRODUCT_LINE_CODE', axis=1, inplace=True)


datasetProduct

,PRODUCT_NUMBER,QUANTITY,UNIT_COST,DAYS_SINCE_INTRODUCTION,1,10,11,12,13,14,...,5,6,7,8,9,1,2,3,4,5
0,1,41770,4.113607,4811,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,10,15928,10.429155,4811,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,100,29980,2.848523,4811,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,101,5764,316.939286,1586,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,102,5014,402.653541,1591,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,95,53086,2.875822,4811,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
111,96,5974,16.269130,4811,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
112,97,3246,24.459522,4062,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
113,98,18362,2.897125,4811,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


Dataset opdelen

In [30]:
x = datasetProduct.drop('QUANTITY', axis = 1)
y = datasetProduct.loc[:,['QUANTITY']]
y

,QUANTITY
0,41770
1,15928
2,29980
3,5764
4,5014
...,...
110,53086
111,5974
112,3246
113,18362


In [40]:
#horizontaal en verticaal snijden
x_train, x_test, y_train, y_test = train_test_split(
    x, y, 
    test_size=0.15, random_state=42
)
y_train

,QUANTITY
105,69996
53,14248
94,13046
112,3246
12,8988
...,...
106,33056
14,3188
92,7436
51,10172


In [32]:
#regressiemodel bouwen
model = LinearRegression().fit(x_train, y_train)
model


LinearRegression()

In [33]:
#predictions maken
y_pred = model.predict(x_test)
y_df = pd.DataFrame(y_pred, columns=['QUANTITY_PREDICTED'])
y_df

,QUANTITY_PREDICTED
0,16059.960783
1,-116.542579
2,35176.402781
3,19783.117053
4,6274.327220
5,33230.225423
6,19619.156818
7,19143.220488
8,30297.754800
9,8360.809872


In [34]:
#combineren
y_test_predictions_merge = pd.concat([y_test.reset_index()['QUANTITY'], y_df], axis=1)
y_test_predictions_merge.loc[y_test_predictions_merge['QUANTITY_PREDICTED'].notna(),:]

,QUANTITY,QUANTITY_PREDICTED
0,4352,16059.960783
1,5014,-116.542579
2,20502,35176.402781
3,11914,19783.117053
4,1506,6274.327220
5,35354,33230.225423
6,8084,19619.156818
7,22840,19143.220488
8,13146,30297.754800
9,11240,8360.809872


Resultaten analyseren

In [35]:
mean_squared_error(y_test['QUANTITY'], y_pred)

96028206.52244134

In [36]:
mean_absolute_error(y_test['QUANTITY'], y_pred)

8174.058251118543